# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

In [ ]:
# both vulnerable. you are sitting East and hold
hand = 'KQ854.854.JT96.8'
seat = 1

# the auction goes:
auction = ["PAD_START", "PAD_START", "1C", "PASS", "1H", "PASS", "3S", "PASS", "4D", "PASS", "4H", "PASS", "4N", "PASS", "5H", "PASS", "6H", "PASS", "PASS", "PASS"]

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# what to lead?
lead_bot = BotLead([True, True], hand, models, sampler, seat, dealer=2, ddsolver=dds, verbose=False)
lead = lead_bot.find_opening_lead(auction)

Loaded lib dds.dll


In [5]:
lead.card

H5

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [6]:
lead.to_dict()['candidates']

[{'card': 'Hx',
  'insta_score': 0.104,
  'expected_tricks_sd': 12.21,
  'p_make_contract': 0.18},
 {'card': 'Sx',
  'insta_score': 0.192,
  'expected_tricks_sd': 12.32,
  'p_make_contract': 0.15},
 {'card': 'SK',
  'insta_score': 0.136,
  'expected_tricks_sd': 12.29,
  'p_make_contract': 0.15},
 {'card': 'Dx',
  'insta_score': 0.174,
  'expected_tricks_sd': 12.24,
  'p_make_contract': 0.12}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [7]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['KQ8xx.8xx.JT9x.8 J.KJxx.KQx.AKQxx T9x.T9.A8xxx.JTx Axxx.AQxx.x.9xxx 0.76808',
 'KQ8xx.8xx.JT9x.8 A.KJxx.KQ8.KQ9xx Txx.T.xxxxx.JTxx J9xx.AQ9xx.A.Axx 0.76433',
 'KQ8xx.8xx.JT9x.8 A.J9xx.AKQ.KQ9xx JTxx.Qx.xxxx.Txx 9xx.AKTx.8x.AJxx 0.76082',
 'KQ8xx.8xx.JT9x.8 x.AJxx.AKx.KQJ9x Axxx.9x.8xx.Txxx JT9.KQTx.Qxx.Axx 0.76043',
 'KQ8xx.8xx.JT9x.8 x.AJTx.KQx.AKQTx JT9x.xx.8xx.9xxx Axx.KQ9x.Axx.Jxx 0.75455',
 'KQ8xx.8xx.JT9x.8 x.AJTx.AKx.KQJxx 9xx.x.Q8xxx.Axxx AJTx.KQ9xx.x.T9x 0.75280',
 'KQ8xx.8xx.JT9x.8 x.Axxx.KQx.AKQJx T9xx.T.A8xx.9xxx AJx.KQJ9x.xx.Txx 0.75228',
 'KQ8xx.8xx.JT9x.8 x.AK9x.KQx.KQJ9x J9xx.x.A8x.Txxxx ATx.QJTxx.xxx.Ax 0.75225',
 'KQ8xx.8xx.JT9x.8 T.J9xx.AKQ.AKJ9x 9xxx.T.xxxx.Qxxx AJx.AKQxx.8x.Txx 0.75147',
 'KQ8xx.8xx.JT9x.8 9.AJxx.KQx.AKQTx Txx.Q9.A8xxx.Jxx AJxx.KTxx.x.9xxx 0.75122',
 'KQ8xx.8xx.JT9x.8 T.AJTx.AKx.KQJxx 9xxx.Q.xxx.T9xxx AJx.K9xxx.Q8x.Ax 0.75022',
 'KQ8xx.8xx.JT9x.8 x.Kxxx.KQx.AKQJx Jxx.AT.8xxx.9xxx AT9x.QJ9x.Ax.Txx 0.74988',
 'KQ8xx.8xx.JT9x.8 x.KJxx.KQ8.AKQ9x Txx.